In [16]:
import sys
import psycopg2


fly_connect = psycopg2.connect(dbname="fly", user="martafd", password="123qwe")
hotel_connect = psycopg2.connect(dbname="hotel", user="martafd", password="123qwe")
account_connect = psycopg2.connect(dbname="account", user="martafd", password="123qwe")

fly_cursor = fly_connect.cursor()
hotel_cursor = hotel_connect.cursor()
account_cursor = account_connect.cursor()

def insert_tables(account_cursor, fly_cursor, hotel_cursor):
    account_cursor.execute("""insert into account (client, amount) values (%s,%s);""", ('Marta', 2000))
    fly_cursor.execute("""insert into fly_booking (client, fly_number, from_city, to_city, date) 
    values(%s,%s,%s,%s,%s);""", ('Marta', 'LRD 34M', 'Rome', 'Lviv', '04/02/2018'))
    hotel_cursor.execute("""insert into hotel_booking (client, hotel, arrival, departure) values(%s,%s,%s,%s);""",
    ('Marta', 'Valery', '04/02/2018', '17/02/2017'))

insert_tables(account_cursor, fly_cursor, hotel_cursor)

def select_all(conn, table):
    cursor = conn.cursor()
    cursor.execute("SELECT * from {0};".format(table))
    conn.commit()
    return cursor.fetchall()

print('Accounts: ', select_all(account_connect, "account"))
print('Fly Bookings: ', select_all(fly_connect, "fly_booking"))
print('Hotel Bookings: ', select_all(hotel_connect, "hotel_booking"))

('Accounts: ', [(1, 'Marta', 2000), (2, 'Marta', 2000)])
('Fly Bookings: ', [(1, 'Marta', 'LRD 34M', 'Rome', 'Lviv', datetime.date(2018, 2, 4)), (2, 'Marta', 'LRD 34M', 'Rome', 'Lviv', datetime.date(2018, 2, 4))])
('Hotel Bookings: ', [(1, 'Marta', 'Valery', datetime.date(2018, 2, 4), datetime.date(2017, 2, 17)), (2, 'Marta', 'Valery', datetime.date(2018, 2, 4), datetime.date(2017, 2, 17))])


In [18]:
fly_connect.tpc_begin(fly_connect.xid(100, 'transaction ID', 'connection 1'))
hotel_connect.tpc_begin(hotel_connect.xid(100, 'transaction ID', 'connection 2'))
account_connect.tpc_begin(account_connect.xid(100, 'transaction ID', 'connection 3'))

try:
    account_connect.cursor().execute("""update account set amount=amount - {} WHERE id=1;""".format(100))
    account_connect.tpc_prepare()
    
    fly_connect.cursor().execute("""insert into fly_booking (client, fly_number, from_city, to_city, date) 
    values(%s,%s,%s,%s,%s);""", ('Dima', 'FY067', 'Lviv', 'Kyiv', '03/05/2018'))
    fly_connect.tpc_prepare()

    hotel_connect.cursor().execute("""insert into hotel_booking (client, hotel, arrival, departure) 
    values(%s,%s,%s,%s);""", ('Dima', 'Figu', '03/05/2018', '15/05/2018'))
    hotel_connect.tpc_prepare()

    # 2 pc
    fly_connect.tpc_commit()
    account_connect.tpc_commit()
    hotel_connect.tpc_commit()
    
except Exception:
    print("Error!! Booking was not done")
    # rollback
    fly_connect.tpc_rollback()
    account_connect.tpc_rollback()
    hotel_connect.tpc_rollback()
    exit()
else:
    print("Booking was done good")

Error!! Booking was not done
